# Intoduction

### ELT Pipeline and Startup Data Analysis

This workbook provides an in-depth analysis of startup data, highlighting success and failure rates across selected countries. Employing the ELT (Extract, Load, Transform) method, the data was first extracted from various sources, loaded into our database, and then transformed to facilitate detailed analysis. SQL queries were utilized to evaluate key metrics such as funding amounts and success rates, calculating success-to-failure ratios rounded to two decimal places.

<div align="center">
  <img src="img/elt_1.png" alt="ELT Pipeline Diagram" style="max-width: 100%; height: auto;">
</div>

The included ELT pipeline diagram visually illustrates this process, showcasing how raw data is systematically processed to derive actionable insights. This structured approach enhances our understanding of startup performance, supports data-driven decision-making, and identifies critical trends.


## Import Required Libraries

In [1]:
import kaggle
import pandas as pd
import sqlalchemy

## STEP-1: Extract

#### Extract Data through Kaggle API

In [2]:
!kaggle datasets download yanmaksi/big-startup-secsees-fail-dataset-from-crunchbase -f "big_startup_secsees_dataset.csv"

Dataset URL: https://www.kaggle.com/datasets/yanmaksi/big-startup-secsees-fail-dataset-from-crunchbase
License(s): Community Data License Agreement - Sharing - Version 1.0
big_startup_secsees_dataset.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


#### Unzip downloaded file

In [3]:
import zipfile
zip_ref = zipfile.ZipFile('big_startup_secsees_dataset.csv.zip')
zip_ref.extractall()
zip_ref.close()

#### Check whether data loaded or not

In [4]:
startup = pd.read_csv("big_startup_secsees_dataset.csv")
startup.head()

,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,/organization/-fame,#fame,http://livfame.com,Media,10000000,operating,IND,16,Mumbai,Mumbai,1,NaN,2015-01-05,2015-01-05
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,DE - Other,Delaware City,2,2014-09-04,2014-03-01,2014-10-14
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,3406878,operating,NaN,NaN,NaN,NaN,1,NaN,2014-01-30,2014-01-30
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008-03-19,2008-03-19
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,-,operating,USA,IL,"Springfield, Illinois",Champaign,1,2010-01-01,2014-07-24,2014-07-24


## STEP-2: Load

#### Load Data to Sql server

#### create a connection with SQL SERVER

In [5]:
from sqlalchemy import create_engine, exc

# Define your connection string
conn_str = (
    'mssql://HAIER-PC\SQLEXPRESS/startup?driver=ODBC+Driver+17+for+SQL+Server'
)

try:
    # Create an engine
    engine = create_engine(conn_str)
    # Test the connection
    conn = engine.connect()
except exc.SQLAlchemyError as e:
    print("Error: Could not make connection to Database")
    print(e)


#### Upload data to SQL SERVER

In [ ]:
startup.to_sql('startup_stage', con=engine, if_exists='append', index=False)


## STEP-3: TRANSFORM

We have uploaded data to sql server and will transform data there.

1.  **Is data loaded or not**

``` SQL
    Select * from startup_stage;
```

output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![Structure](img/data_load.png)

2.  **Check for Table Schema**

``` 
    SELECT COLUMN_NAME, DATA_TYPE
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = 'startup_stage';
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![Structure](img/schema.png)

3. **Check for Duplicates**

``` SQL
    select name, count(*) as dulpicate
    from startup_stage
    group by name
    having count(*) >1;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![duplicates](img/duplicates.png)

4. **Remove Duplicates**

``` SQL
    WITH CTE AS (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY name ORDER BY CASE WHEN name IS NULL THEN 1 ELSE 0 END) AS row_num
        FROM startup_stage
    )
    -- Delete rows where row number is greater than 1, i.e., duplicates with empty data
    DELETE FROM CTE
    WHERE row_num > 1;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![duplicates drop](img/drs.png)

5. **Count unique Startups**

```SQL
    SELECT COUNT(DISTINCT name) AS unique_count
    FROM startup_stage;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![unique](img/unique_1.png)

6. **Basic Statistics**

```SQL
    SELECT 
        SUM(funding_total_usd) AS total_funding,
        AVG(funding_total_usd) AS average_funding,
        MIN(funding_total_usd) AS min_funding,
        MAX(funding_total_usd) AS max_funding
    FROM startup_stage;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![unique](img/stat_1.png)

7. **Top 10 Average Funding Amounts Across Startups**

``` SQL
    SELECT TOP 10
        name,
        AVG(funding_total_usd) AS avg_funding_amount
    FROM startup_stage
    GROUP BY name
    order by avg_funding_amount desc;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![Top 10](img/top.png)

8. **Distribution of funding amounts by range**

```SQL
    SELECT 
        CASE 
            WHEN funding_total_usd < 1000000 THEN 'Under 1M'
            WHEN funding_total_usd BETWEEN 1000000 AND 5000000 THEN '1M to 5M'
            WHEN funding_total_usd BETWEEN 5000000 AND 10000000 THEN '5M to 10M'
            ELSE 'Over 10M'
        END AS funding_range,
        COUNT(*) AS count_startups
    FROM startup_stage
    GROUP BY 
        CASE 
            WHEN funding_total_usd < 1000000 THEN 'Under 1M'
            WHEN funding_total_usd BETWEEN 1000000 AND 5000000 THEN '1M to 5M'
            WHEN funding_total_usd BETWEEN 5000000 AND 10000000 THEN '5M to 10M'
            ELSE 'Over 10M'
        END;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![Top 10](img/distribution_1.png)

9. **Count of startups by category**

```SQL
    SELECT Top 10
    category_list, COUNT(*) AS count_startups 
    FROM startup_stage
    GROUP BY category_list
    ORDER BY count_startups DESC;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![Top 10](img/countcc.png)

#### Feature Engineering

1. **Funding amount per Round**

```SQL
    SELECT TOP 10
        name,
        funding_total_usd / funding_rounds AS funding_per_round
    FROM startup_stage;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![Top 10](img/per.png)

2.  **Time between funding rounds**

```SQL
    SELECT TOP 10
        name,
        DATEDIFF(day, first_funding_at, last_funding_at) / (funding_rounds - 1) AS avg_days_between_rounds
    FROM startup_stage
    WHERE funding_rounds > 1;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![Top 10](img/time_bw.png)

3. **Count of startups by status for each country**

```SQL
    SELECT Top 10
        country_code,
        status,
        COUNT(*) AS count_startups
    FROM startup_stage
    GROUP BY country_code, status
    ORDER BY country_code, status;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![Top 10](img/top_10.png)

4. **Select specific country and count startups by status**

```SQL
    SELECT 
        country_code,
        status,
        COUNT(*) AS count_startups
    FROM startup_stage
    WHERE country_code IN ('PAK', 'IND', 'USA')
    GROUP BY country_code, status
    ORDER BY country_code, status;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![Top 10](img/ip.png)

1.  **Calculate success to failure rate  for given countries**

```SQL
WITH status_counts AS (
    SELECT 
        country_code,
        status,
        COUNT(*) AS count_startups
    FROM startup_stage
    WHERE country_code IN ('PAK', 'IND', 'USA')
    GROUP BY country_code, status
),
success_failure_counts AS (
    SELECT
        country_code,
        SUM(CASE WHEN status IN ('operating', 'ipo') THEN count_startups ELSE 0 END) AS success_count,
        SUM(CASE WHEN status IN ('closed', 'acquired') THEN count_startups ELSE 0 END) AS failure_count
    FROM status_counts
    GROUP BY country_code
)
SELECT
    country_code,
    success_count,
    failure_count,
    CASE 
        WHEN failure_count > 0 THEN ROUND(success_count * 1.0 / failure_count, 2)
        ELSE NULL
    END AS success_to_failure_rate
FROM success_failure_counts
ORDER BY country_code;
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;![Top 10](img/stf.png)